In [1]:
!pip install -e . & cd /data1/courses/2024-2025/4343SADL6/Tumor_Group/SiNGR-tumor

In [1]:
import torch

/data1/courses/2024-2025/4343SADL6/Tumor_Group/conda_envs/sdl_course/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print('GPU available?', torch.cuda.is_available())


GPU available? True


# Troubleshooting GeoDesic Transform

In [1]:
import os
import click
from pathlib import Path

import nibabel as nib
import numpy as np
import cv2
import torch
from natsort import natsorted

import mlpipeline
# from mlpipeline import *
# from mlpipeline import utils
from mlpipeline.utils import *
import mlpipeline.utils.common as utils
from mlpipeline.utils.geodesic_map import write_data4d, merge_seg, get_data


/data1/courses/2024-2025/4343SADL6/Tumor_Group/conda_envs/sdl_course/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# def transform_geo_dist(dist, gt, label_name):
#     print("Transforming Geo Dist")
#     print("Dist shape is", dist.shape)
#     print("Gt Shape is", gt.shape)
#     margin = 0.5

#     def _transform_geo_dist_layer(dist_layer, gt_layer):
#         print("Dist Layer Shape is ", dist_layer.shape)
#         print("GT Layer Shape is ", gt_layer.shape)
#         fg = (gt_layer == 1)
#         bg = (gt_layer == 0)
#         dist_layer[fg] = torch.clamp_min(dist_layer[fg], min=0.0)
#         dist_layer[bg] = torch.clamp_max(dist_layer[bg], max=0.0)

#         if torch.sum(fg) == 0:
#             return torch.full_like(dist_layer, -1)

#         fg_max = dist_layer[fg].max()
#         dist_layer[bg] = torch.clamp_min(dist_layer[bg], min=-fg_max)

#         dist_layer[fg] = dist_layer[fg] / (fg_max + 1e-8)
#         dist_layer[bg] = dist_layer[bg] / (fg_max + 1e-8)

#         if label_name in ["fast_sgc_margin"]:
#             dist_layer[fg] = (dist_layer[fg] / 1.0) * (1.0 - margin) + margin
#             dist_layer[bg] = (dist_layer[bg] / 1.0) * (1.0 - margin) - margin

#         dist_layer = torch.clamp(dist_layer, min=-1.0, max=1.0)

#         if label_name == "fast_sgc_clamp":
#             dist_layer[(gt_layer > 0) & (dist_layer <= 0)] = dist_layer[dist_layer > 0.0001].min()
#             dist_layer[(gt_layer == 0) & (dist_layer >= 0)] = dist_layer[dist_layer < -0.0001].max()

#         return dist_layer

#     assert not torch.any(torch.isnan(dist))
#     for index in range(0, dist.shape[-1]):
#         dist[..., index] = _transform_geo_dist_layer(dist[..., index], gt[..., index])
#     assert not torch.any(torch.isnan(dist))
#     return dist

# Original code

In [ ]:
def transform_geo_dist(dist, gt, label_name):
    # print("Transforming Geo Dist")
    # print("Dist shape is", dist.shape)
    # print("Gt Shape is", gt.shape)
    margin = 0.5

    def _transform_geo_dist_layer(dist_layer, gt_layer):
        # print("Dist Layer Shape is ", dist_layer.shape)
        # print("GT Layer Shape is ", gt_layer.shape)
        fg = (gt_layer == 1)
        bg = (gt_layer == 0)
        dist_layer[fg] = torch.clamp_min(dist_layer[fg], min=0.0)
        dist_layer[bg] = torch.clamp_max(dist_layer[bg], max=0.0)

        if torch.sum(fg) == 0:
            return torch.full_like(dist_layer, -1)

        fg_max = dist_layer[fg].max()
        dist_layer[bg] = torch.clamp_min(dist_layer[bg], min=-fg_max)

        dist_layer[fg] = dist_layer[fg] / (fg_max + 1e-8)
        dist_layer[bg] = dist_layer[bg] / (fg_max + 1e-8)

        if label_name in ["fast_sgc_margin"]:
            dist_layer[fg] = (dist_layer[fg] / 1.0) * (1.0 - margin) + margin
            dist_layer[bg] = (dist_layer[bg] / 1.0) * (1.0 - margin) - margin

        dist_layer = torch.clamp(dist_layer, min=-1.0, max=1.0)

        if label_name == "fast_sgc_clamp":
            dist_layer[(gt_layer > 0) & (dist_layer <= 0)] = dist_layer[dist_layer > 0.0001].min()
            dist_layer[(gt_layer == 0) & (dist_layer >= 0)] = dist_layer[dist_layer < -0.0001].max()

        return dist_layer

    assert not torch.any(torch.isnan(dist))

    # For each segmentation channel (label), apply transformation
    for ch in range(gt.shape[-1]):
        for z in range(dist.shape[-1]):
            dist[:, :, z] = _transform_geo_dist_layer(dist[:, :, z], gt[:, :, z, ch])

    assert not torch.any(torch.isnan(dist))
    return dist


In [10]:
def transform_gd_dist(dist, gt):
    def _transform_gd_dist_layer(dist_layer, gt_layer):
        fg = (gt_layer == 1)
        bg = (gt_layer == 0)

        dist_layer[fg] = torch.clamp_min(dist_layer[fg], min=0.52)
        dist_layer[bg] = torch.clamp_max(dist_layer[bg], max=0.48)
        return dist_layer

    assert not torch.any(torch.isnan(dist))
    # for index in range(0, dist.shape[-1]):
    #     dist[..., index] = _transform_geo_dist_layer(dist[..., index], gt[..., :, index])
    for index in range(0, gt.shape[-1]):
        dist[..., index] = _transform_geo_dist_layer(dist[..., index], gt[..., :, index])
    assert not torch.any(torch.isnan(dist))
    return dist

In [21]:
def run_transformation(label_name, dataset, root_dir, gt_dir, output_dir):
    # sub_dirs = natsorted(list(Path(root_dir).glob("*")))
    sub_dirs = natsorted([p for p in Path(root_dir).glob("*") if p.is_dir()])
    root_name = Path(root_dir).stem
    print(root_dir, len(sub_dirs))

    for si, sub_dir in enumerate(sub_dirs):
        paths = [p for p in Path(sub_dir).glob("*.nii") if not p.name.endswith("_seg.nii")]
        sub_dir_name = sub_dir.name
        # print("Sub Dir Name is ",sub_dir_name)
        os.makedirs(Path(output_dir) / sub_dir_name, exist_ok=True)

        hard_gt_path = Path(gt_dir) / sub_dir_name / f"{sub_dir_name}_seg.nii"
        # print("Hard GT Path is ", hard_gt_path)
        if dataset == "BRATS":
            hard_gt = np.transpose(merge_seg(hard_gt_path), [3, 2, 1, 0])
            print("Transposing Hard GT")
        else:
            hard_gt = get_data(hard_gt_path, is_seg=True)[0]
            hard_gt = np.transpose(np.expand_dims(hard_gt, axis=0), [3, 2, 1, 0])
        hard_gt = torch.tensor(hard_gt, device="cuda:0")

        for path in paths:
            print("Path is (inside output generation) ", path)
            name = path.stem.replace(".nii", "")
            raw_geo_gt = nib.load(path).get_fdata()

            geo_gt = torch.tensor(raw_geo_gt, device="cuda:0")
            if label_name == "gd_normed":
                geo_gt = transform_gd_dist(geo_gt, hard_gt)
            else:
                geo_gt = transform_geo_dist(geo_gt, hard_gt, label_name)

            geo_gt = geo_gt.cpu().numpy()
            print("geo_gt shape before transpose:", geo_gt.shape)
            geo_gt = geo_gt[np.newaxis, ...]
            permuted_geo_gt = np.transpose(geo_gt, [3, 2, 1, 0])
            output_name = name.replace(f"_{root_name}_", f"_{label_name}_")
            output_path = Path(output_dir) / sub_dir_name / f"{output_name}.nii.gz"
            print("Output Path is ", output_path)
            write_data4d(permuted_geo_gt, output_path)
    return

In [ ]:
# @click.command()
# @click.option("--label_name", default="fast_sgc_margin")
# @click.option("--dataset", default="BRATS")
# @click.option("--root_dir", default="../BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData")
# @click.option("--gt_dir", default="../BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData")
# @click.option("--output_dir", default="../BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData")

In [4]:
def main(label_name: str, dataset: str, root_dir: str, gt_dir: str, output_dir: str):
    temp_dir = "./temp"
    print(root_dir)
    print(output_dir)
    print(gt_dir)

    os.makedirs(temp_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)
    print("Running Transformation Now")
    run_transformation(label_name, dataset, root_dir, gt_dir, output_dir)


In [5]:
root_dir = "/data1/courses/2024-2025/4343SADL6/Tumor_Group/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"
gt_dir = "/data1/courses/2024-2025/4343SADL6/Tumor_Group/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"
output_dir = "/data1/courses/2024-2025/4343SADL6/Tumor_Group/Outputs/GeodesicTransform_Output"
label_name = "fast_sgc_margin"
dataset = "BRATS"

In [ ]:
# python SiNGR-tumor/mlpipeline/utils/geodesic_transform.py --label_name "fast_sgc_margin" --dataset "BRATS" 
# --root_dir /data1/courses/2024-2025/4343SADL6/Tumor_Group/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData 
# --gt_dir /data1/courses/2024-2025/4343SADL6/Tumor_Group/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData 
# --output_dir /data1/courses/2024-2025/4343SADL6/Tumor_Group/Outputs/GeodesicTransform_Output

In [23]:
if __name__ == "__main__":
    main(label_name, dataset, root_dir, gt_dir, output_dir)

/data1/courses/2024-2025/4343SADL6/Tumor_Group/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData
/data1/courses/2024-2025/4343SADL6/Tumor_Group/Outputs/GeodesicTransform_Output
/data1/courses/2024-2025/4343SADL6/Tumor_Group/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData
Running Transformation Now
/data1/courses/2024-2025/4343SADL6/Tumor_Group/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData 369
Transposing Hard GT
Path is (inside output generation)  /data1/courses/2024-2025/4343SADL6/Tumor_Group/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t1.nii


: 

: 

: 